In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import pickle

import tensorflow as tf
from tensorflow.keras import models
from tensorflow import keras
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator, image
from keras.applications.xception import Xception, preprocess_input, decode_predictions
from keras.applications.inception_v3 import InceptionV3,preprocess_input

In [ ]:
if not os.path.exists('./train'):
    os.mkdir('./train')
if not os.path.exists('./train/cup'):
    os.mkdir('./train/cup')     
if not os.path.exists('./train/fork'):
    os.mkdir('./train/fork')     
if not os.path.exists('./train/glass'):
    os.mkdir('./train/glass')     
if not os.path.exists('./train/knife'):
    os.mkdir('./train/knife')     
if not os.path.exists('./train/plate'):
    os.mkdir('./train/plate')     
if not os.path.exists('./train/spoon'):
    os.mkdir('./train/spoon')     
    
if not os.path.exists('./test'):
    os.mkdir('./test')
    
trainList = pd.read_csv('train.csv')

In [ ]:
for img in os.listdir('./images'):
    
    if (not os.path.exists(f'./train/{img}')) and (not(os.path.exists(f'./test/{img}'))):
        imgName = int(img.split('.')[0])
        
        if imgName in trainList.Id.values:
            
            imgLabel = trainList[trainList.Id == imgName].label.values
            
            if imgLabel == 'cup':
                shutil.copy(f'./images/{img}', f'./train/cup/{img}')
            elif imgLabel == 'fork':
                shutil.copy(f'./images/{img}', f'./train/fork/{img}')
            elif imgLabel == 'glass':
                shutil.copy(f'./images/{img}', f'./train/glass/{img}')
            elif imgLabel == 'knife':
                shutil.copy(f'./images/{img}', f'./train/knife/{img}')
            elif imgLabel == 'plate':
                shutil.copy(f'./images/{img}', f'./train/plate/{img}')
            elif imgLabel == 'spoon':
                shutil.copy(f'./images/{img}', f'./train/spoon/{img}')

        else:            
            
            shutil.copy(f'./images/{img}', f'./test/{img}')
            
totalTrainImgs = len(os.listdir("./train/cup")) + len(os.listdir("./train/fork")) + \
                len(os.listdir("./train/glass")) + len(os.listdir("./train/knife")) + \
                len(os.listdir("./train/plate")) + len(os.listdir("./train/spoon"))

totalTestImgs = len(os.listdir("./test"))

print(f'There is a total of {totalTrainImgs} images in the training set')
print(f'There is a total of {totalTestImgs} images in the test set')            

In [ ]:
dataGenerator = ImageDataGenerator(rescale=1/255, validation_split=0.2)    

trainGenerator = dataGenerator.flow_from_directory(directory='./train',
                                                     batch_size=32,
                                                     target_size=(300, 300), 
                                                     subset="training",
                                                     shuffle=True,
                                                     class_mode='categorical')

valGenerator = dataGenerator.flow_from_directory(directory='./train',
                                                   batch_size=32,
                                                   target_size=(300, 300),
                                                   subset="validation",
                                                   shuffle=True,
                                                   class_mode='categorical')

In [ ]:
#optimizer = SGD(learning_rate=0.1, momentum=0.4)
learning_rate = 0.0007
optimizer = Adam(learning_rate=learning_rate)
#loss = CategoricalCrossentropy(from_logits=True)
loss = CategoricalCrossentropy()

model = models.Sequential([
    
    layers.Conv2D(8, (7,7), activation='relu', input_shape=(300,300,3)),
    layers.MaxPool2D(4,4),
    layers.Conv2D(16, (7,7), activation='relu'),
    layers.MaxPool2D(4,4),
    layers.Conv2D(16, (5,5), activation='relu'),
    layers.MaxPool2D(2,2),
    layers.Dropout(0.2),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPool2D(2,2),
    layers.Dropout(0.2),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(6, activation='softmax')
])

model.summary()

In [ ]:
model.compile(loss = loss, optimizer=optimizer, metrics=['accuracy'])

In [ ]:
history = model.fit( trainGenerator, epochs=10, validation_data=valGenerator)

In [ ]:
######################################

In [ ]:
pixels = 500

dataGenerator = ImageDataGenerator(preprocessing_function=preprocess_input,
                                    validation_split=0.2,
                                    #rotation_range=30,
                                    #shear_range=0.2,
                                    #zoom_range=0.3,
                                    #vertical_flip=True
                                    #brightness_range=(20,40),
                                    #featurewise_std_normalization=True
                                  )    

trainGenerator = dataGenerator.flow_from_directory(directory='./train',
                                                     batch_size=32,
                                                     target_size=(pixels, pixels), 
                                                     subset="training",
                                                     shuffle=True,
                                                     class_mode='categorical')

valGenerator = dataGenerator.flow_from_directory(directory='./train',
                                                   batch_size=16,
                                                   target_size=(pixels, pixels),
                                                   subset="validation",
                                                   shuffle=True,
                                                   class_mode='categorical')

In [ ]:
base_model = tf.keras.applications.Xception(weights='imagenet',
                                            include_top=False,
                                            input_shape=(300, 300, 3))

base_model.trainable = False


In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint('xception_v1_{epoch:02d}_{val_accuracy:.3f}.h5',
                                             save_best_only=True,
                                             save_weights_only=True,
                                             monitor='val_accuracy',
                                             mode='max')

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=10,
    mode="auto",
    min_lr=0.0001,
)

In [ ]:
inputs = keras.Input(shape=(300,300,3))
base = base_model(inputs, training=False)

vectors = layers.GlobalAveragePooling2D()(base)

inner1 = layers.Dense(128, activation='relu')(vectors)
drop1 = layers.Dropout(0.3)(inner1)
#inner2 = layers.Dense(128, activation='relu')(drop1)
#drop2 = layers.Dropout(0.3)(inner2)
outputs = layers.Dense(6, activation='softmax')(drop1)

modelx = Model(inputs, outputs)
optimizer = Adam(learning_rate=0.001)
loss = CategoricalCrossentropy()
modelx.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
modelx.summary()

In [ ]:
history = modelx.fit(trainGenerator,
                     validation_data=valGenerator,
                     steps_per_epoch=50,
                     epochs=20,
                     callbacks=[checkpoint],
                     #validation_steps=5,
                     #verbose=2
                    )

In [ ]:
##########################

In [ ]:
def GetModel(modelName):
    
    basemodel_Xce = tf.keras.applications.Xception(weights='imagenet',
                                                   include_top=False,
                                                   input_shape=(pixels, pixels, 3))

    basemodel_Inc = tf.keras.applications.InceptionV3(weights='imagenet',
                                                      include_top=False,
                                                      input_shape=(pixels, pixels, 3))
    
    basemodel_EffB7 = tf.keras.applications.EfficientNetB7(weights='imagenet',
                                                       include_top=False,
                                                       input_shape=(pixels, pixels, 3)) 

    basemodel_Res50V2 = tf.keras.applications.resnet_v2.ResNet50V2(weights='imagenet',
                                                                 include_top=False,
                                                                 input_shape=(pixels, pixels, 3)
                                                                  ) 

    basemodel_VGG16 = tf.keras.applications.vgg16.VGG16(weights='imagenet',
                                                          include_top=False,
                                                          input_shape=(pixels, pixels, 3),
                                                       ) 
    
   
    basemodels = {'Xception': basemodel_Xce,
                  'InceptionV3': basemodel_Inc,
                  'efficientnetB7': basemodel_Eff,
                  'ResNet50V2': basemodel_Res50V2,
                  'VGG16' : basemodel_VGG16
                 }
    
    return basemodels[modelName]

In [ ]:
def MakeModel(modelName = 'Xception', learning_rate = 0.001, size_inner=128, droprate=0.3):
    
    base_model = GetModel(modelName)
    
    base_model.trainable = False

    inputs = Input(shape=(pixels, pixels, 3))
    base = base_model(inputs, training=False)
    vectors = layers.GlobalAveragePooling2D()(base)
    
    inner = layers.Dense(size_inner, activation='relu')(vectors)
    drop = layers.Dropout(droprate)(inner)
    
    outputs = layers.Dense(6, activation='softmax')(drop)
    model = Model(inputs, outputs)
    
    #########################################################
    
    optimizer = Adam(learning_rate=learning_rate)
    loss = CategoricalCrossentropy()
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
    return model

In [ ]:
scores = {}
#lr = 0.001
size = 128
droprate = 0.3
learningRates = [0.0005, 0.0007, 0.001, 0.003, 0.005, 0.01]

for lr in learningRates:
    print(lr)
    model = MakeModel(modelName='Xception', learning_rate=lr)
    history = model.fit(trainGenerator, validation_data=valGenerator, steps_per_epoch=50, epochs=15)
    scores[lr] = history.history
    print('\n\n')

In [ ]:
for lr , hist in scores.items():
    plt.plot(hist['val_accuracy'], label=lr)
    
plt.xticks(np.arange(15))
plt.ylim(0.85,0.98)
plt.legend()

In [ ]:
del scores[0.005, 0.01]

In [ ]:
for lr , hist in scores.items():
    plt.plot(hist['val_accuracy'], label=lr)
    
plt.xticks(np.arange(15))
plt.ylim(0.94,0.98)
plt.legend()

In [ ]:
#########################################################

In [ ]:
def GetCheckpoint(modelName):
    checkpoint = keras.callbacks.ModelCheckpoint(modelName + '_{epoch:02d}_{val_accuracy:.3f}.h5',
                                                 save_best_only=True,
                                                 #save_weights_only=True,
                                                 monitor='val_accuracy',
                                                 mode='max')
    return checkpoint
#plateau = tf.keras.callbacks.ReduceLROnPlateau(
#    monitor="val_loss",
#    factor=0.1,
#    patience=10,
#    mode="auto",
#    min_lr=0.0001,
#)

In [ ]:
models = ['Xception', 'InceptionV3', 'efficientnetB7', 'ResNet50V2', 'VGG16']

In [ ]:
scores = {}
lr = 0.001
size = 128
droprate = 0.3
modelName = 'ResNet50V2'

checkpoint = GetCheckpoint(modelName)


model = MakeModel(modelName=modelName, learning_rate=lr, size_inner=size, droprate=droprate)
history = model.fit(trainGenerator, validation_data=valGenerator, steps_per_epoch=50, epochs=15, callbacks=[checkpoint])

In [ ]:
scores = {}
lr = 0.01
size = 128
droprate = 0.3
modelName = 'VGG16'

checkpoint = GetCheckpoint(modelName)

model = MakeModel(modelName=modelName, learning_rate=lr, size_inner=size, droprate=droprate)
history = model.fit(trainGenerator, validation_data=valGenerator, steps_per_epoch=50, epochs=15, callbacks=[checkpoint])

In [ ]:
scores = {}
lr = 0.001
size = 128
droprate = 0.3

for modelName in models:
    checkpoint = GetCheckpoint(modelName)
    model = MakeModel(modelName=modelName, learning_rate=lr, size_inner=size, droprate=droprate)
    history = model.fit(trainGenerator, validation_data=valGenerator, steps_per_epoch=50, epochs=15, callbacks=[checkpoint])
   

In [ ]:
newModel = model = makeModel(modelName='Xception', learning_rate=lr, size_inner=size, droprate=droprate)
newModel.load_weights('xception_v1_17_0.966.h5')

In [ ]:
base_model = InceptionV3(include_top=False,
                         input_shape=(300, 300, 3),
                        weights = 'imagenet')

base_model.trainable = False

#base_model.summary()

In [ ]:
from tensorflow.keras import Model

#inputs = keras.Input(shape=(300,300,3))
#base = base_model(inputs, training=False)

last_layer_out = base_model.get_layer('mixed7')
last_output = last_layer_out.output

x = layers.Flatten()(last_output)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(6, activation='softmax')(x)

#inner = keras.layers.Dense(128, activation='relu')(vectors)
#drop = keras.layers.Dropout(0.2)(inner)
#outputs = keras.layers.Dense(10)(drop)
#model = keras.Model(inputs, outputs)

modelx2 = Model(base_model.input, x)
#modelx = Model(inputs, outputs)

optimizer = Adam(learning_rate=0.001)
loss = CategoricalCrossentropy()

modelx2.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
modelx2.summary()

In [ ]:
history = modelx2.fit(trainGenerator,
                     validation_data=valGenerator,
                     steps_per_epoch=50,
                     epochs=20,
                     callbacks=[reduce_lr, model_checkpoint],
                     #validation_steps=5,
                     #verbose=2
                    )

In [ ]:
newModel = makeModel(modelName='Xception', learning_rate=lr, size_inner=size, droprate=droprate)
newModel.load_weights('xception_v1_09_0.973.h5')

In [ ]:
count =0
res = []
classes = trainGenerator.class_indices
labels = dict((v, k) for k, v in classes.items())

for file in os.listdir('./test'):
    imgPath = './test/' + file
    img = image.load_img(imgPath, target_size=(pixels, pixels))
    x = np.array(img)
    X = np.array([x])
    X = preprocess_input(X)
    preds = model.predict(X)

    label = labels[np.argmax(preds[0])]
    Id = file.split('.')[0]
    res.append([Id, label])
    if count%200==0:
        print(count, Id, label)
    count += 1

In [ ]:
results = pd.DataFrame(res, columns=['Id', 'label'])
results

results.to_csv('submission_MAC_Resnet50.csv', index=False)